### -- Question 1 - The period during which the buying and selling prices of PNJ and SJC gold fluctuate the most.

In [2]:
import pandas as pd 
from sqlalchemy import create_engine

# Đọc dữ liệu từ tệp CSV vào DataFrame 
df = pd.read_csv(r"data/ten_year.csv")

# Tạo kết nối tới database SQLite trong bộ nhớ
engine = create_engine('sqlite://', echo=False)

# Lưu DataFrame vào database
df.to_sql('gold_prices', engine, index=False, if_exists='replace')

# Định nghĩa truy vấn SQL (đã điều chỉnh để sử dụng MAX và MIN chỉ khi phù hợp)
query = """ 
WITH PriceChanges AS (
    SELECT type, buy, sell, date,
        LAG(buy) OVER (PARTITION BY type ORDER BY date) AS prev_buy_price,
        LAG(sell) OVER (PARTITION BY type ORDER BY date) AS prev_sell_price,
        LEAD(buy) OVER (PARTITION BY type ORDER BY date) AS next_buy_price,
        LEAD(sell) OVER (PARTITION BY type ORDER BY date) AS next_sell_price
    FROM gold_prices
),
MaxChanges AS (
    SELECT type, MAX(ABS(next_buy_price - prev_buy_price)) AS max_buy_price_change, MAX(ABS(next_sell_price - prev_sell_price)) AS max_sell_price_change
    FROM PriceChanges
    GROUP BY type
)
SELECT pc.type, mc.max_buy_price_change, mc.max_sell_price_change,
    MIN(CASE WHEN ABS(pc.next_buy_price - pc.prev_buy_price) = mc.max_buy_price_change THEN pc.date END) AS max_buy_price_change_date,
    MIN(CASE WHEN ABS(pc.next_sell_price - pc.prev_sell_price) = mc.max_sell_price_change THEN pc.date END) AS max_sell_price_change_date
FROM PriceChanges pc
JOIN MaxChanges mc ON pc.type = mc.type
GROUP BY pc.type, mc.max_buy_price_change, mc.max_sell_price_change;
"""

# Thực hiện truy vấn SQL trên DataFrame 
result = pd.read_sql_query(query, engine)

# Hiển thị kết quả 
print(result)


  type  max_buy_price_change  max_sell_price_change max_buy_price_change_date  \
0  PNJ                   5.4                    5.4                2020-08-12   
1  SJC                 506.2                  318.9                2020-07-27   

  max_sell_price_change_date  
0                 2020-08-12  
1                 2016-10-10  


### Question 2 - The fluctuations in the buying and selling prices of PNJ and SJC on the day with the most updates.

In [3]:
import pandas as pd 
from sqlalchemy import create_engine

# Đọc dữ liệu từ tệp CSV vào DataFrame 
df = pd.read_csv(r"data/ten_year.csv")

# Tạo kết nối tới database SQLite trong bộ nhớ
engine = create_engine('sqlite://', echo=False)

# Lưu DataFrame vào database
df.to_sql('gold_prices', engine, index=False, if_exists='replace')

# Định nghĩa truy vấn SQL (đã điều chỉnh để sử dụng MAX và MIN chỉ khi phù hợp)
query = """ 
WITH update_count AS (
    SELECT date, type, COUNT(*) AS update_count
    FROM gold_prices
    GROUP BY date, type
)
SELECT uc.date, uc.type, uc.update_count,
    MAX(CASE WHEN uc.type = 'PNJ' THEN gp.buy END) AS max_buy_PNJ,
    MIN(CASE WHEN uc.type = 'PNJ' THEN gp.sell END) AS min_sell_PNJ,
    MAX(CASE WHEN uc.type = 'SJC' THEN gp.buy END) AS max_buy_SJC,
    MIN(CASE WHEN uc.type = 'SJC' THEN gp.sell END) AS min_sell_SJC
FROM update_count uc
JOIN gold_prices gp ON uc.date = gp.date AND uc.type = gp.type
WHERE uc.date = (
    SELECT date
    FROM update_count
    ORDER BY update_count DESC
    LIMIT 1
)
GROUP BY uc.date, uc.type, uc.update_count
ORDER BY uc.date, uc.type;
"""
# Thực hiện truy vấn SQL trên DataFrame 
result = pd.read_sql_query(query, engine)

# Hiển thị kết quả 
print(result)


         date type  update_count  max_buy_PNJ  min_sell_PNJ  max_buy_SJC  \
0  2022-02-24  PNJ             7         55.7          54.2          NaN   
1  2022-02-24  SJC            18          NaN           NaN         66.9   

   min_sell_SJC  
0           NaN  
1          63.5  


### Question 3 - Comparison of the stability of buying and selling prices between PNJ and SJC.

In [5]:
import pandas as pd 
from sqlalchemy import create_engine

# Đọc dữ liệu từ tệp CSV vào DataFrame 
df = pd.read_csv(r"data/ten_year.csv")

# Tạo kết nối tới database SQLite trong bộ nhớ
engine = create_engine('sqlite://', echo=False)

# Lưu DataFrame vào database
df.to_sql('gold_prices', engine, index=False, if_exists='replace')

# Định nghĩa truy vấn SQL (đã điều chỉnh để sử dụng MAX và MIN chỉ khi phù hợp)
query = """ 
SELECT 
    type,
    AVG(buy) AS avg_buy,
    AVG(sell) AS avg_sell,
    SQRT(SUM((buy - (SELECT AVG(buy) FROM gold_prices)) * (buy - (SELECT AVG(buy) FROM gold_prices))) / COUNT(*)) AS stdev_buy,
    SQRT(SUM((sell - (SELECT AVG(sell) FROM gold_prices)) * (sell - (SELECT AVG(sell) FROM gold_prices))) / COUNT(*)) AS stdev_sell
FROM gold_prices
GROUP BY type;
"""



# Thực hiện truy vấn SQL trên DataFrame 
result = pd.read_sql_query(query, engine)

# Hiển thị kết quả 
print(result)


  type    avg_buy   avg_sell  stdev_buy  stdev_sell
0  PNJ  44.074809  43.379192  11.297830   11.016432
1  SJC  47.880957  47.348254  15.909679   14.908578
